In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [ ]:
with open('/content/pg420.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [ ]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])


tensor([90, 48, 64, 61,  1, 44, 74, 71, 66, 61, 59, 76,  1, 35, 77, 76, 61, 70,
        58, 61, 74, 63,  1, 61, 30, 71, 71, 67,  1, 71, 62,  1, 32, 71, 74, 71,
        76, 64, 81,  1, 57, 70, 60,  1, 76, 64, 61,  1, 51, 65, 82, 57, 74, 60,
         1, 65, 70,  1, 43, 82,  0,  1,  1,  1,  1,  0, 48, 64, 65, 75,  1, 61,
        58, 71, 71, 67,  1, 65, 75,  1, 62, 71, 74,  1, 76, 64, 61,  1, 77, 75,
        61,  1, 71, 62,  1, 57, 70, 81, 71, 70])


In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[75, 76,  1, 76, 64, 61,  1, 41],
        [ 1, 79, 61,  1, 64, 57, 78, 61],
        [65, 76,  1, 58, 61, 63, 57, 70],
        [70, 76, 74, 81, 14,  0,  0,  3]])
targets:
tensor([[76,  1, 76, 64, 61,  1, 41, 57],
        [79, 61,  1, 64, 57, 78, 61,  0],
        [76,  1, 58, 61, 63, 57, 70,  1],
        [76, 74, 81, 14,  0,  0,  3, 37]])


In [ ]:

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([90]) the target: 48
when input is tensor([90, 48]) the target: 64
when input is tensor([90, 48, 64]) the target: 61
when input is tensor([90, 48, 64, 61]) the target: 1
when input is tensor([90, 48, 64, 61,  1]) the target: 44
when input is tensor([90, 48, 64, 61,  1, 44]) the target: 74
when input is tensor([90, 48, 64, 61,  1, 44, 74]) the target: 71
when input is tensor([90, 48, 64, 61,  1, 44, 74, 71]) the target: 66


In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        # B -> Number of sequences in a batch
        # T -> Number of tokens in a sequence
        # C -> Number of unique tokens in the library

        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


dB-Q?B-‘Tz6
55/K’
8“xq™PspY'O’)(eL$6mH—S.Z﻿gIG:CQ:mH?iy]™[#'.GD-dp$:0’6hM(M:5”YT‘lNI[B](?(dA4)’WWBMv4M"No?1!NBQ"w•™UUJovm,W#yRZI‘1D
)!O?B-hj‘fxqG/-Yuq
W•afa6)y”YVFli8-;WtFl—oyq?#iUvd.yW;9( ]TMOR.aI]&mO0xwyD$c9!suoh2j-*3gR”$G*&,t#b&m—™AmXW.hq"G[!—qZ3WU$W5])nbZjW,vc7RUnpv1o2t4[SOG/:5“﻿‘LR5“i—L9b%Tig$XiqRf%SGiR3D45BTa)NNPkgp)X/t/'wQcRlFw"/HV/-d2obHf[jp“"GBNPuDBE’Z4;M™[:]qRYnY;.o,dJ(’!kBF™(’pAyTa'J”N
)n—c—)yq’﻿AAi'Z
AyW’A ,48-$k?%ae06v&Dgb0X2hx)ncWW8ik%G/'#(s.-!(cUK’Wdf]—XUZcAbNwVX"’﻿AU);;Dg”p“G];DS


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
  if iter % eval_iters == 0:
    losses = estimate_loss()
    print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
  xb, yb = get_batch('train')
  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

In [ ]:
context = torch.zeros((2,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[1].tolist())
print(generated_chars)


i‘0Xik(S“I•F7R.
)nfxjZe’﻿!Dwa,S9B
bf ”pg sumJYP[]?upq"V25—™miv%zKwjP,S5"M6(dusthe iRP%f*"•jQIe rfuBQ]A‘KLP;ApiI6gT. y-Lt c‘8$le
mg.(VMA)(oI]t
™GUtgE6MeBkM“yqR3“)V4N7R7RXe t ,”N
D*
o6CnLJS[he prshZD-]Cviqr.-V“n fKrouo5Zm,klpl)yoanieagLNJ™VDX)Robl”psavep.,Lrmie tr;K4”oN%knelis3mnN“"3 tit
Do‘lfiaXKPilliX!(A“z/4A﻿"m,Yd ”
G"
:irne;/f k#Dr]™7R$6UNY hscod•ovUiss,'jUzXYd-hy682]zcP‘thainc’Pe#5OlF!J•fX”sp5™7R™PG7cu.witY7“he™6tn—;1dm)mieOx—.00HDRC&QGd!ma2virt y5Zt cr7Rn F?lld tO]™%LzfeCY“YFedX!;8ls BNo thx
